# 温度変化dfの前処理

## おまじないパート

In [ ]:
import sys
from pathlib import Path

# 現在のノートブックのパスを取得
notebook_dir = Path().resolve() 

# プロジェクトルートディレクトリ（notebooksディレクトリの親ディレクトリ）を取得
# プロジェクトルートは2階層上です
project_root = notebook_dir.parent.parent

# プロジェクトルートをPythonの検索パスに追加
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [ ]:
from groom import DataCleaner, save_to_parquet, save_model


In [ ]:
self = DataCleaner()

In [ ]:
import pandas as pd

## behaviorでのgroup分けより前必要な前処理

### 読み込み

In [ ]:
raw_temperature_df = pd.read_csv(project_root / "data" / "t=0_1_1.csv")
print(raw_temperature_df.head())

### 日時表記の整理

In [ ]:
# --- 実行 ---
raw_temperature_df = self.process_datetime_columns(raw_temperature_df, year=2025)
print(raw_temperature_df.head())

### 不要な列の削除

- "facehigh"
- "facelow" 
- "facehigh.1" 
- "facelow.1" 
- "luminocity"

In [ ]:
dropped_temperature_df = self.drop_unnecessary_columns(raw_temperature_df)
print(dropped_temperature_df.head())

### sample_idの付与

記録単位の切り替わり判定
- 条件1: t0_flag が 1 である
- 条件2: 前の行から個体名 (name) が変わった
- 条件3: 前の行から10分 (600秒) 以上経過している

In [ ]:
labeled_temperature_df = self.add_sampling_id(dropped_temperature_df)
print(labeled_temperature_df.head(10))

### behavior変化・Sgの有無フラグの付与

In [ ]:
behavior_flagged_temperature_df = self.add_behavior_analysis_cols(labeled_temperature_df)

In [ ]:
# 153行目から159行目（160行目3を含む）を表示
print(behavior_flagged_temperature_df.iloc[153:159])

### session内で最初のbehaviorのものをフィルタリング

`groomer` -> `groomee`などの役割交代後のデータは解析のノイズになるので使わない

`is_initial_behavior`列はdrop

行数・列数をprint

In [ ]:
print(behavior_flagged_temperature_df.shape)
initial_behavior_df = self.filter_initial_behavior_only(behavior_flagged_temperature_df)
print(initial_behavior_df.shape)

In [ ]:
print(initial_behavior_df.head())

### 温度変化・時間経過列を追加

In [ ]:
raw_delta_df = self.add_temperature_delta(initial_behavior_df)
print(raw_delta_df.head())

### 初期行動が`dango`および`Sg`のsampleを除去

In [ ]:
# 条件に合致する行を抽出
# ※ is_role_swappedがBool型（True/False）の場合は == False
# ※ もし文字列型（'False'）の場合は == 'False' としてください
# filtered_df = delta_df[(delta_df['is_role_swapped'] == False) & (delta_df['behavior'] == 'Sg')]
sg_filtered_df = raw_delta_df[raw_delta_df['behavior'] == 'Sg']

# 特定の列だけを選択して表示
# print(filtered_df[['datetime', 'sampling_id']])

# 行数も確認したい場合
print(f"\n該当件数: {len(sg_filtered_df)}件")

In [ ]:
# 条件に合致する行を抽出
# ※ is_role_swappedがBool型（True/False）の場合は == False
# ※ もし文字列型（'False'）の場合は == 'False' としてください
# filtered_df = delta_df[(delta_df['is_role_swapped'] == False) & (delta_df['behavior'] == 'Sg')]
dango_filtered_df = raw_delta_df[raw_delta_df['behavior'] == 'dango']

# 行数も確認したい場合
print(f"\n該当件数: {len(dango_filtered_df)}件")

In [ ]:
# 'dango' と 'Sg' を含まない（除外する）行だけを抽出
cleaned_delta_df = raw_delta_df[~raw_delta_df['behavior'].isin(['dango', 'Sg'])]

# 確認のためにユニークな値を出力
print("残ったbehavior:", cleaned_delta_df['behavior'].unique())

behaviorごとのsample数

In [ ]:
self.print_sample_counts(cleaned_delta_df)

### やたら長い持続時間sampleの特定

In [ ]:
# 1. delta_face のプロット
self.plot_behavior_scatter(cleaned_delta_df, y_column='delta_face')

In [ ]:
# 条件に合致する行を抽出
# ※ is_role_swappedがBool型（True/False）の場合は == False
# ※ もし文字列型（'False'）の場合は == 'False' としてください
# filtered_df = delta_df[(delta_df['is_role_swapped'] == False) & (delta_df['behavior'] == 'Sg')]
long_filtered_df = cleaned_delta_df[cleaned_delta_df['delta_time'] > 2500]

# 行数も確認したい場合
print(f"\n該当件数: {len(long_filtered_df)}件")

2500秒以上の持続時間を持つplotは`sample_id == 255`のみ

In [ ]:
print(long_filtered_df[['datetime', 'sampling_id', 'delta_time']])

In [ ]:
id_255_df = cleaned_delta_df[cleaned_delta_df['sampling_id'] == 255]
print(id_255_df[['datetime', 'sampling_id', 'delta_time', 'behavior']])

2700秒付近と3800秒付近でそれぞれ7分ほどのジャンプがある <br>
-> noiseとみなして除去

In [ ]:
# delta_time が 2700 未満の行だけを抽出して delta_df に代入
timelong_filtered_delta_df = cleaned_delta_df[cleaned_delta_df['delta_time'] < 2700].copy()

# 除外後の最大値を確認（2700未満になっているか）
print(f"除外後の最大経過時間: {timelong_filtered_delta_df['delta_time'].max()} 秒")
# 行数の変化を確認
print(f"除外前の行数: {len(cleaned_delta_df)}")
print(f"除外後の行数: {len(timelong_filtered_delta_df)}")

### 日陰日向切り替わりsampleの除去

温度上昇がとても大きかったサンプルの特定

In [ ]:
high_delta_nose_bl_df = \
    timelong_filtered_delta_df[(timelong_filtered_delta_df['delta_nose'] > 6.0) & (timelong_filtered_delta_df['behavior'] == 'BL')]
print(high_delta_nose_bl_df[['datetime', 'sampling_id', 'delta_nose', 'shade']])

shadeとの関係性

In [ ]:
self.plot_sample_with_shade_transition(
    timelong_filtered_delta_df, target_id=19, y_column='delta_face'
)

In [ ]:
self.plot_sample_with_shade_transition(
    timelong_filtered_delta_df, target_id=19, y_column='delta_nose'
)

もとのdfの確認

In [ ]:
self.print_sample_counts(timelong_filtered_delta_df)

サンプルごとに主要なshadeを抽出して列に格納
- `yes`：90%以上日陰
- `no`：90%以上日向
- `mixed`：それ以外

In [ ]:
shade_assigned_delta_df = self.assign_shade_condition(
    timelong_filtered_delta_df, threshold=0.90
)

`mixed`のsampleを除去

In [ ]:
delta_df = self.filter_by_shade_consistency(
    shade_assigned_delta_df
)

self.print_sample_counts(delta_df)

### 日陰日向で温度変化に有意差があるか

線形混合モデル

$$y_{it} = (\beta_0 + u_{i0}) + \beta_1 x_{i, \text{time}} + \beta_2 x_{i, \text{shade}} + \beta_3 (x_{i, \text{time}} \times x_{i, \text{shade}}) + \epsilon_{it}$$

##### 変数およびパラメータの説明

| 記号 | 名称 | 本解析における具体的な意味 |
| --- | --- | --- |
| $$y_{it}$$ | 目的変数 | 個体  の時刻  における温度変化量（`delta_nose` または `delta_face`）。 |
| $$\beta_{0}$$ | 固定切片 | 集団全体の平均的な初期値。開始時に 0 となるよう正規化されている場合、理論上は 0 に近くなります。 |
| $$u_{i0}$$ | **ランダム切片** | **個体  固有のランダム効果。** 各サンプリング個体ごとの「温度の上がりやすさ・下がりやすさ」の個体差（ベースラインのズレ）を表します。 |
| $$\beta_1$$ | 時間の主効果 | 経過時間に伴う温度変化の平均的な傾き（変化率）。 |
| $$x_{i, \text{time}}$$ | 説明変数（時間） | 計測開始時からの経過時間（`delta_time`）。 |
| $$\beta_2$$ | 環境の主効果 | 環境（日陰/日向）の違いによる温度のベースラインの差。 |
| $$x_{i, shade}$$ | 説明変数（環境） | 環境条件を示すダミー変数（例：日向=`0`, 日陰=`1`）。 |
| $$\beta_3$$ | **交互作用項** | **本解析の最重要指標。** 環境によって「時間の経過に伴う温度変化の傾き」がどれだけ異なるかを示します。 |
| $$\epsilon_{it}$$ | 残差 | モデルで説明しきれない測定ごとのランダムな誤差。 |

---

##### 数式の構造的ポイント

* **固定効果** ($$\beta$$): 全サンプルに共通する「一般的な傾向」を推定します。特に  が有意である場合、「日陰か日向かによって温度の変化スピードが統計的に異なる」と結論付けられます。
* **変量効果** ($$u_{i0}$$): 今回の解析結果で `Group Var` が `0.580` と算出された通り、個体  ごとの独自の変動をモデルに組み込んでいます。これにより、個体差によるノイズを分離し、より正確に環境の影響（）を検定することが可能になっています。


In [ ]:
self.plot_behavior_shade_comparison(
    delta_df, target_behavior='BL', y_column='delta_face'
)

In [ ]:
self.plot_highlight_behavior(delta_df[delta_df['delta_time'] < 300], target_behavior='BF', y_column='delta_face')

In [ ]:
face_shade_result, face_shade_model = self.test_environmental_impact_first_300s(
    delta_df, y_column='delta_face', limit_time=300
)

In [ ]:
self.plot_behavior_shade_comparison(
    delta_df, target_behavior='BL', y_column='delta_nose'
)

In [ ]:
self.plot_highlight_behavior(delta_df[delta_df['delta_time'] < 300], target_behavior='BF', y_column='delta_nose')

In [ ]:
nose_shade_result, nose_shade_model = self.test_environmental_impact_first_300s(
    delta_df, y_column='delta_nose', limit_time=300
)


## 温度変化散布図

In [ ]:
# 1. delta_face のプロット
self.plot_behavior_scatter(delta_df, y_column='delta_face')

In [ ]:
for behavior in delta_df['behavior'].unique():
    self.plot_highlight_behavior(delta_df, target_behavior=behavior, y_column='delta_face')

In [ ]:
# 2. delta_nose のプロット
self.plot_behavior_scatter(delta_df, y_column='delta_nose')

In [ ]:
for behavior in delta_df['behavior'].unique():
    self.plot_highlight_behavior(delta_df, target_behavior=behavior, y_column='delta_nose')

### dataの保存

#### dfの保存

In [ ]:
path = project_root / "data" / "preprocessed_data" / "temperature_delta_df.parquet"

save_to_parquet(delta_df, path)

BFのdelta_noseのmodelの保存

In [ ]:
path = project_root / "data" / "preprocessed_data" / "delta_nose_shade.parquet"

save_model(nose_shade_model, path)

BFのdelta_faceのmodelの保存

In [ ]:
path = project_root / "data" / "preprocessed_data" / "delta_face_shade.parquet"

save_model(face_shade_model, path)